In [1]:
from math import ceil, pi, e
from sympy import series, Symbol, Rational, diff, factorial, fraction, simplify, solveset, S, ln, expand, N, symbols
from sympy.functions import sin, cos, tan
import pandas as pd
from IPython.display import display, HTML
from termcolor import colored

In [12]:
def truncate(floatt):
  '''truncate floatt to n digits'''
  return round(round(floatt,10)*100000)/100000.0

def createtable(coords, values, length, xs, algo):
  for i in range(len(coords)-1):
      temp = list()
      orders = values[-1]
      for j in range(len(orders)-1):
        temp.append(truncate(orders[j+1]-orders[j]))
      values.append(temp)

  # creating table
  data = {'x':xs,'y':values[0]}
  if algo=='backward':    
    for i in range(1,length):
      [values[i].insert(0,'-') for j in range(length - len(values[i]))]
  [data.update({i:values[i]}) for i in range(1,length)]

  df = pd.DataFrame.from_dict(data, orient='index')
  df = df.transpose().fillna('-') # table created
  display(HTML(df.to_html(index=False)))

def choosealgo(coords, x, derivtimes, symbs):
  xs = [i[0] for i in coords]
  length = len(xs)
  distance = list()

  distance.append(abs(x-xs[0]))
  distance.append(abs(x-xs[-1]))
  if length%2!=0:
    distance.append(abs(x-xs[length//2]))

  indx = distance.index(min(distance))
  if indx==0:
    print(colored('Newton Forward Difference','red'))
    newtonforwdivideddiff(coords, xs, x, derivtimes, symbs)
  elif indx==1:
    print(colored('Newton Backward Difference','red'))
    newtonbackdivideddiff(coords, xs, x, derivtimes, symbs)
  elif indx==2:
    print(colored('Stirling Central Difference','red'))
    stirlingcentdiff(coords, xs, x, derivtimes, symbs)
  print(colored('=================== (VALUE DONE) ===================', 'red'))

In [13]:
def newtonforwdivideddiff(coords, xs, x, derivtimes, symbs):
  values = [[i[1] for i in coords]]
  length = len(coords)
  h = truncate(xs[1] - xs[0])
  s = truncate((x- xs[0])/h)
  print(f's = ({x}-{xs[0]})/({xs[1]}-{xs[0]})\ns = {s}')

  createtable(coords, values, length, xs, 'forward')
  for i in range(derivtimes):
    findforwardequation(values, s, length, h, i+1, symbs)
    

def findforwardequation(values, s, length, h, derivtimes, symbs):
  string = list()
  Px = list()
  vals = list()
  apos = "'"*derivtimes
  times = f'**{derivtimes}'
  x = Symbol('s')

  print(colored('===== RED CAN BE SKIPPED AS THEY HAVE 0 VALUE','red'))
  for i in range(1,length):
    temp2 = x
    for j in range(1,i):
      temp2*=(x-j)
    vals.append(expand(diff(temp2,x,derivtimes)))
    temp2 = f'({vals[-1]}) *△{i}{symbs[1]}0/{i}!'
    if values[i][-1]==0 or vals[-1]==0:
      temp2 = colored(temp2,'red')
    string.append(temp2)
    temp2 = f'({vals[-1]})({values[i][-1]})/{i}!'
    if values[i][-1]==0 or vals[-1]==0:
      temp2 = colored(temp2,'red')
    Px.append(temp2)

  print(f'\nP{apos}{length-1}s =\t(1/h{times})*\n', end='')
  [print(f'\t({i}) +\n ',end='') for i in string]

  print(f'\nP{apos}{length-1}s =\t(1/{h}{times})*\n', end='')
  [print(f'\t({i}) +\n',end='') for i in Px]

  sums = list()
  print(f'\nP{apos}{length-1}s =\t(1/{h**derivtimes})*\n', end='')
  for i in range(len(Px)):
    try:
      sums.append(truncate(vals[i].subs(x,s)*values[i+1][0]/factorial(i+1)))
    except:
      sums.append(truncate(vals[i].subs(x,s)*values[i+1][0]/factorial(i+1)))
    print(f'\t({sums[-1]}) +\n',end='')
    
  print(f'\nP{apos}{length-1}s =\t{sum(sums)/(h**derivtimes)}\n')

In [14]:
def newtonbackdivideddiff(coords, xs, x, derivtimes, symbs):
  values = [[i[1] for i in coords]]
  length = len(coords)
  h = truncate(xs[1] - xs[0])
  s = truncate((x- xs[-1])/h)
  print(f's = ({x}-{xs[-1]})/({xs[1]}-{xs[0]})\ns = {s}')

  createtable(coords, values, length, xs, 'backward')
  for i in range(derivtimes):
    findbackwardequation(values, s, length, h, i+1, symbs)
    

def findbackwardequation(values, s, length, h, derivtimes, symbs):
  string = list()
  Px = list()
  vals = list()
  apos = "'"*derivtimes
  times = f'**{derivtimes}'
  x = Symbol('s')

  print(colored('===== RED CAN BE SKIPPED AS THEY HAVE 0 VALUE','red'))
  for i in range(1,length):
    temp2 = x
    for j in range(1,i):
      temp2*=(x+j)
    vals.append(expand(diff(temp2,x,derivtimes)))
    temp2 = f'({vals[-1]}) *▽{i}{symbs[1]}{length-1}/{i}!'
    if values[i][-1]==0 or vals[-1]==0:
      temp2 = colored(temp2,'red')
    string.append(temp2)
    temp2 = f'({vals[-1]})({values[i][-1]})/{i}!'
    if values[i][-1]==0 or vals[-1]==0:
      temp2 = colored(temp2,'red')
    Px.append(temp2)

  print(f'\nP{apos}{length-1}s =\t(1/h{times})*\n', end='')
  [print(f'\t({i}) +\n ',end='') for i in string]

  print(f'\nP{apos}{length-1}s =\t(1/{h}{times})*\n', end='')
  [print(f'\t({i}) +\n',end='') for i in Px]

  sums = list()
  print(f'\nP{apos}{length-1}s =\t(1/{h**derivtimes})*\n', end='')
  for i in range(len(Px)):
    try:
      sums.append(truncate(vals[i].subs(x,s)*values[i+1][-1]/factorial(i+1)))
    except:
      sums.append(truncate(vals[i].subs(x,s)*values[i+1][-1]/factorial(i+1)))
    print(f'\t({sums[-1]}) +\n',end='')
    
  print(f'\nP{apos}{length-1}s =\t{sum(sums)/(h**derivtimes)}\n')

In [15]:
def findmid(val):
  n = len(val) 
  mid = n//2    
  if n % 2 == 0: 
      median = (val[mid] + val[mid - 1])/2
  else: 
      median = val[mid]
  return median

def stirlingcentdiff(coords, xs, x, derivtimes,symbs):
  values = [[i[1] for i in coords]]
  length = len(coords)
  h = truncate(xs[1] - xs[0])
  s = truncate((x - findmid(xs))/h)
  print(f's = ({x}-{findmid(xs)})/({xs[1]}-{xs[0]})\ns = {s}')

  for i in range(len(coords)-1):
    temp = list()
    orders = values[-1]
    for j in range(len(orders)-1):
      temp.append(truncate(orders[j+1]-orders[j]))
    values.append(temp)

  createtable(coords, values, length, xs, 'stirling')
  for i in range(derivtimes):
    findstirlingequation(values, s, length, h, i+1, symbs)
  
def findstirlingequation(values, s, length, h, derivtimes, symbs):
  a = Symbol('s')
  ylist = [findmid(values[0])]
  slist = [1,a]
  apos = "'"*derivtimes
  times = f'**{derivtimes}'

  for i in range(1,length):
    ylist.append(findmid(values[i]))
    if i % 2 == 0:
      slist.append(slist[-2]*(a**2 - (i//2)**2))
    else:
      slist.append(slist[-1]*a)
  slist = slist[:-1]
  stringg = [expand(diff(i,a,derivtimes)) for i in slist]
  
  print(colored('===== RED CAN BE SKIPPED AS THEY HAVE 0 VALUE','red'))

  print(f'\nP{length-1}s =\t({ylist[0]}) +')
  for i in range(1,length):
    if ylist[i]==0:
      print(colored(f'\t({ylist[i]})({slist[i]})/{i}! +','red'))
    else:
      print(f'\t({ylist[i]})({slist[i]})/{i}! +')
    
  print(f'\nP{apos}{length-1}s =\t(1/h{times})*\n\t({ylist[0]}) +')
  for i in range(1,length):
    val = stringg[i]
    if ylist[i]==0 or val==0:
      print(colored(f'\t({ylist[i]})({val})/{i}! +','red'))
    else:
      print(f'\t({ylist[i]})({val})/{i}! +')
      
  print(f'\nP{apos}{length-1}s =\t(1/{h}{times})*\n\t({ylist[0]}) +')
  for i in range(1,length):
    val = truncate(stringg[i].subs(a,s))
    if ylist[i]==0 or val==0:
      print(colored(f'\t({ylist[i]})({val})/{i}! +','red'))
    else:
      print(f'\t({ylist[i]})({val})/{i}! +')

  sumlist = [ylist[0]]
  print(f'\nP{apos}{length-1}s =\t(1/{h**derivtimes})*\n\t({ylist[0]}) +')
  for i in range(1,length):
    val = truncate(ylist[i]*truncate(stringg[i].subs(a,s))/factorial(i))
    sumlist.append(val)
    if ylist[i]==0 or val==0:
      print(colored(f'\t({val}) +','red'))
    else:
      print(f'\t({val}) +')

  print(f'\nP{apos}{length-1}s =\t{truncate(sum(sumlist)/h**derivtimes)}')

In [18]:
# indexes are part values for question
# derivtimes define how many times we need to differntiate Px
coords = [(0,0),(25,32),(50,58),(75,78),(100,92),(125,100)]
#coords = [(0.1,0.003),(0.3,0.067),(0.5,0.148),(0.7,0.248),(0.9,0.370),(1.1,0.518),(1.3,0.698)]
xvals = [25, 50, 90, 0.65]
derivtimes = [1, 1, 2, 2]
x, y = symbols('t x')

# set part num to solve for that part only (where partnum = xvals index to choose its value)
partnum = 0
choosealgo(coords, xvals[partnum], derivtimes[partnum], (x,y))  

Newton Forward Difference
s = (25-0)/(25-0)
s = 1.0


x,y,1,2,3,4,5
0.0,0.0,32,-6,0,0,0
25.0,32.0,26,-6,0,0,-
50.0,58.0,20,-6,0,-,-
75.0,78.0,14,-6,-,-,-
100.0,92.0,8,-,-,-,-
125.0,100.0,-,-,-,-,-


===== RED CAN BE SKIPPED AS THEY HAVE 0 VALUE

P'5s =	(1/h**1)*
	((1) *△1x0/1!) +
 	((2*s - 1) *△2x0/2!) +
 	((3*s**2 - 6*s + 2) *△3x0/3!) +
 	((4*s**3 - 18*s**2 + 22*s - 6) *△4x0/4!) +
 	((5*s**4 - 40*s**3 + 105*s**2 - 100*s + 24) *△5x0/5!) +
 
P'5s =	(1/25.0**1)*
	((1)(8.0)/1!) +
	((2*s - 1)(-6.0)/2!) +
	((3*s**2 - 6*s + 2)(0.0)/3!) +
	((4*s**3 - 18*s**2 + 22*s - 6)(0.0)/4!) +
	((5*s**4 - 40*s**3 + 105*s**2 - 100*s + 24)(0.0)/5!) +

P'5s =	(1/25.0)*
	(32.0) +
	(-3.0) +
	(0.0) +
	(0.0) +
	(0.0) +

P'5s =	1.16

=================== (VALUE DONE) ===================
